##### Copyright 2019 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 네트워크

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/8_networks_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     View on TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/8_networks_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Run in Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/8_networks_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     View source on GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/8_networks_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a></td>
</table>

## 소개

In this colab we will cover how to define custom networks for your agents. The networks help us define the model that is trained by agents. In TF-Agents you will find several different types of networks which are useful across agents:

**주요 네트워크**

- **QNetwork**: Used in Qlearning for environments with discrete actions, this network maps an observation to value estimates for each possible action.
- **CriticNetworks**: Also referred to as `ValueNetworks` in literature, learns to estimate some version of a Value function mapping some state into an estimate for the expected return of a policy. These networks estimate how good the state the agent is currently in is.
- **ActorNetworks**: Learn a mapping from observations to actions. These networks are usually used by our policies to generate actions.
- **ActorDistributionNetworks**: Similar to `ActorNetworks` but these generate a distribution which a policy can then sample to generate actions.

**Helper Networks**

- **EncodingNetwork** : 사용자는 네트워크의 입력에 적용 할 전처리 레이어의 매핑을 쉽게 정의 할 수 있습니다.
- **DynamicUnrollLayer** : 시간 순서에 따라 적용되는 에피소드 경계에서 네트워크 상태를 자동으로 재설정합니다.
- **ProjectionNetwork**: Networks like `CategoricalProjectionNetwork` or `NormalProjectionNetwork` take inputs and generate the required parameters to generate Categorical, or Normal distributions.

All examples in TF-Agents come with pre-configured networks. However these networks are not setup to handle complex observations.

If you have an environment which exposes more than one observation/action and you need to customize your networks then this tutorial is for you!

## 설정

tf-agent를 아직 설치하지 않은 경우 다음을 실행하십시오.

In [ ]:
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import random_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.networks import encoding_network
from tf_agents.networks import network
from tf_agents.networks import utils
from tf_agents.specs import array_spec
from tf_agents.utils import common as common_utils
from tf_agents.utils import nest_utils

tf.compat.v1.enable_v2_behavior()

## Defining Networks

### 네트워크 API

In TF-Agents we subclass from Keras [Networks](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/keras/engine/network.py). With it we can:

- 대상 네트워크를 만들 때 필요한 복사 작업을 단순화합니다.
- Perform automatic variable creation when calling `network.variables()`.
- Validate inputs based on network input_specs.

##EncodingNetwork As mentioned above the `EncodingNetwork` allows us to easily define a mapping of pre-processing layers to apply to a network's input to generate some encoding.

The EncodingNetwork is composed of the following mostly optional layers:

- 전처리 레이어
- 전처리 결합기
- 전환
- 반음 낮추다
- 밀집한

The special thing about encoding networks is that input preprocessing is applied. Input preprocessing is possible via `preprocessing_layers` and `preprocessing_combiner` layers.  Each of these can be specified as a nested structure. If the `preprocessing_layers` nest is shallower than `input_tensor_spec`, then the layers will get the subnests. For example, if:

```
input_tensor_spec = ([TensorSpec(3)] * 2, [TensorSpec(3)] * 5) preprocessing_layers = (Layer1(), Layer2())
```

then preprocessing will call:

```
preprocessed = [preprocessing_layers[0](observations[0]),                 preprocessing_layers[1](obsrevations[1])]
```

However if

```
preprocessing_layers = ([Layer1() for _ in range(2)],                         [Layer2() for _ in range(5)])
```

전처리는 다음을 호출합니다.

```python
preprocessed = [   layer(obs) for layer, obs in zip(flatten(preprocessing_layers),                                     flatten(observations)) ]
```


### Custom Networks

To create your own networks you will only have to override the `__init__` and `call` methods. Let's create a custom network using what we learned about `EncodingNetworks` to create an ActorNetwork that takes observations which contain an image and a vector.


In [ ]:
class ActorNetwork(network.Network):

  def __init__(self,
               observation_spec,
               action_spec,
               preprocessing_layers=None,
               preprocessing_combiner=None,
               conv_layer_params=None,
               fc_layer_params=(75, 40),
               dropout_layer_params=None,
               activation_fn=tf.keras.activations.relu,
               enable_last_layer_zero_initializer=False,
               name='ActorNetwork'):
    super(ActorNetwork, self).__init__(
        input_tensor_spec=observation_spec, state_spec=(), name=name)

    # For simplicity we will only support a single action float output.
    self._action_spec = action_spec
    flat_action_spec = tf.nest.flatten(action_spec)
    if len(flat_action_spec) > 1:
      raise ValueError('Only a single action is supported by this network')
    self._single_action_spec = flat_action_spec[0]
    if self._single_action_spec.dtype not in [tf.float32, tf.float64]:
      raise ValueError('Only float actions are supported by this network.')

    kernel_initializer = tf.keras.initializers.VarianceScaling(
        scale=1. / 3., mode='fan_in', distribution='uniform')
    self._encoder = encoding_network.EncodingNetwork(
        observation_spec,
        preprocessing_layers=preprocessing_layers,
        preprocessing_combiner=preprocessing_combiner,
        conv_layer_params=conv_layer_params,
        fc_layer_params=fc_layer_params,
        dropout_layer_params=dropout_layer_params,
        activation_fn=activation_fn,
        kernel_initializer=kernel_initializer,
        batch_squash=False)

    initializer = tf.keras.initializers.RandomUniform(
        minval=-0.003, maxval=0.003)

    self._action_projection_layer = tf.keras.layers.Dense(
        flat_action_spec[0].shape.num_elements(),
        activation=tf.keras.activations.tanh,
        kernel_initializer=initializer,
        name='action')

  def call(self, observations, step_type=(), network_state=()):
    outer_rank = nest_utils.get_outer_rank(observations, self.input_tensor_spec)
    # We use batch_squash here in case the observations have a time sequence
    # compoment.
    batch_squash = utils.BatchSquash(outer_rank)
    observations = tf.nest.map_structure(batch_squash.flatten, observations)

    state, network_state = self._encoder(
        observations, step_type=step_type, network_state=network_state)
    actions = self._action_projection_layer(state)
    actions = common_utils.scale_to_spec(actions, self._single_action_spec)
    actions = batch_squash.unflatten(actions)
    return tf.nest.pack_sequence_as(self._action_spec, [actions]), network_state

Let's create a `RandomPyEnvironment` to generate structured observations and validate our implementation.

In [ ]:
action_spec = array_spec.BoundedArraySpec((3,), np.float32, minimum=0, maximum=10)
observation_spec =  {
    'image': array_spec.BoundedArraySpec((16, 16, 3), np.float32, minimum=0,
                                        maximum=255),
    'vector': array_spec.BoundedArraySpec((5,), np.float32, minimum=-100,
                                          maximum=100)}

random_env = random_py_environment.RandomPyEnvironment(observation_spec, action_spec=action_spec)

# Convert the environment to a TFEnv to generate tensors.
tf_env = tf_py_environment.TFPyEnvironment(random_env)

Since we've defined the observations to be a dict we need to create preprocessing layers to handle these.

In [ ]:
preprocessing_layers = {
    'image': tf.keras.models.Sequential([tf.keras.layers.Conv2D(8, 4),
                                        tf.keras.layers.Flatten()]),
    'vector': tf.keras.layers.Dense(5)
    }
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
actor = ActorNetwork(tf_env.observation_spec(), 
                     tf_env.action_spec(),
                     preprocessing_layers=preprocessing_layers,
                     preprocessing_combiner=preprocessing_combiner)

Now that we have the actor network we can process observations from the environment.

In [ ]:
time_step = tf_env.reset()
actor(time_step.observation, time_step.step_type)

이 같은 전략을 사용하여 에이전트가 사용하는 기본 네트워크를 사용자 정의 할 수 있습니다. 전처리를 정의하고 나머지 네트워크에 연결할 수 있습니다. 사용자 정의를 정의 할 때 네트워크의 출력 계층 정의가 일치하는지 확인하십시오.